In [2]:
import pandas as pd
from typing import Dict, Any, Optional
from sqlalchemy import create_engine
import json
import os

def extract_data():
    """ Extract data of the leukemia patients from the database in PostgreSQL."""

    try:
        os.chdir("../../Leukemia-Cancer-Risk-ETL")
    except FileNotFoundError:
        print("""
            FileNotFoundError - Puede que el directorio no exista o no estas ubicad@ en la ruta especificada.
            """)
    os.chdir("..")
    print(os.getcwd())

    with open("/home/user/leucemia/Leukemia-Cancer-Risk-ETL/credentials.json", "r", encoding = "utf-8") as f:
        credentials = json.load(f)

    db_host = credentials["db_host"]
    db_name = credentials["db_name"]
    db_user = credentials["db_user"]
    db_password = credentials["db_password"]
    
    engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}")
    query = "SELECT * FROM leukemia_clean_data_positive"
    df = pd.read_sql(query, engine)
    
    return df

In [3]:
import pandas as pd
from sqlalchemy import create_engine
import json
import os

def extract_data():
    """ Extract data of the leukemia patients from the database in PostgreSQL."""

    try:
        os.chdir("/home/user/leucemia/Leukemia-Cancer-Risk-ETL/")
    except FileNotFoundError:
        print("""
            FileNotFoundError - Puede que el directorio no exista o no estas ubicad@ en la ruta especificada.
            """)
    os.chdir("..")
    print(os.getcwd())

    with open("Leukemia-Cancer-Risk-ETL/credentials.json", "r", encoding = "utf-8") as f:
        credentials = json.load(f)

    db_host = credentials["db_host"]
    db_name = credentials["db_name"]
    db_user = credentials["db_user"]
    db_password = credentials["db_password"]
    
    engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}")
    query = "SELECT * FROM leukemia_clean_data_positive"
    df = pd.read_sql(query, engine)
    
    return df

In [4]:
extract_data()



/home/user/leucemia


,id,age,gender,country,wbc_count,rbc_count,platelet_count,hemoglobin,bone_marrow_blasts,genetic_mutation,...,radiation_exposure,infection_history,bmi,chronic_illness,immune_disorders,ethnicity,socioeconomic_status,urban_rural,leukemia_status,living_status
0,2,15,Female,China,4857,4.81,277877,11.9,97,1,...,0,0,28.7,0,0,A,Low,Urban,1,0
1,7,87,Male,Brazil,8858,4.81,134683,11.0,83,0,...,0,0,31.6,0,0,C,High,Urban,1,0
2,14,53,Male,India,4434,5.43,271727,11.4,35,0,...,0,0,26.7,0,1,A,High,Urban,1,1
3,16,88,Male,Brazil,2780,4.08,271071,14.9,70,1,...,0,0,18.5,0,1,A,Medium,Urban,1,0
4,22,21,Male,Canada,6322,5.69,264846,13.1,14,0,...,0,0,28.1,0,1,A,Low,Urban,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21374,143150,8,Female,Russia,8182,5.08,275330,10.3,37,0,...,0,1,23.2,1,1,B,Low,Urban,1,0
21375,143158,59,Female,Italy,6362,4.27,269952,9.9,12,0,...,0,0,26.2,0,0,C,Low,Urban,1,1
21376,143163,6,Male,South Korea,6402,5.14,247872,9.6,0,0,...,0,0,29.5,0,0,A,Low,Rural,1,0
21377,143171,63,Male,USA,6422,5.05,318880,11.1,19,0,...,0,1,26.4,0,0,A,Medium,Rural,1,1


In [5]:
import pandas as pd
from typing import Dict, Any

In [6]:
def extract_medical_history(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts and transforms data for MedicalHistory dimension
    
    Args:
        df: Source DataFrame containing raw data
        
    Returns:
        Processed DataFrame for Dim_MedicalHistory
    """
    medical_history = df[[
        'genetic_mutation', 'family_history', 'smoking_status',
        'alcohol_consumption', 'radiation_exposure', 'infection_history',
        'chronic_illness', 'immune_disorders'
    ]].copy().rename(columns={
        'genetic_mutation': 'Genetic_Mutation',
        'family_history': 'Family_History',
        'smoking_status': 'Smoking_Status',
        'alcohol_consumption': 'Alcohol_Consumption',
        'radiation_exposure': 'Radiation_Exposure',
        'infection_history': 'Infection_History',
        'chronic_illness': 'Chronic_Illness',
        'immune_disorders': 'Immune_Disorders'
    })
    
    medical_history = medical_history.drop_duplicates().fillna('Unknown')
    medical_history.reset_index(drop=True, inplace=True)
    medical_history['MedicalHistoryId'] = medical_history.index + 1
    
    return medical_history[['MedicalHistoryId'] + medical_history.columns[:-1].tolist()]

In [19]:
def extract_region(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts and transforms data for Region dimension
    
    Args:
        df: Source DataFrame containing raw data
        
    Returns:
        Processed DataFrame for Dim_Region
    """
    region = df[['Country']].copy().drop_duplicates()
    region.reset_index(drop=True, inplace=True)
    region['RegionId'] = region.index + 1
    return region[['RegionId', 'Country']]

In [ ]:
def extract_medical_history(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts and transforms data for MedicalHistory dimension
    using your actual column names.
    """
    medical_history = df[[
        'Genetic_Mutation', 'Family_History', 'Smoking_Status',
        'Alcohol_Consumption', 'Radiation_Exposure', 'Infection_History',
        'Chronic_Illness', 'Immune_Disorders'
    ]].copy()
    

    medical_history.reset_index(drop=True, inplace=True)
    medical_history['MedicalHistoryId'] = medical_history.index + 1
    
    return medical_history[['MedicalHistoryId'] + medical_history.columns[:-1].tolist()]

In [29]:
def extract_leukemia_facts(df: pd.DataFrame, patient_ids: Dict[Any, int], region_ids: Dict[str, int]) -> pd.DataFrame:
    """Extracts facts using your actual column names"""
    facts = df[[
        'Patient_ID', 'Country', 'WBC_Count', 'RBC_Count', 'Platelet_Count',
        'Hemoglobin_Level', 'Bone_Marrow_Blasts', 'BMI', 'Leukemia_Status'
    ]].copy().rename(columns={
        'Patient_ID': 'PatientId',
        'Hemoglobin_Level': 'Hemoglobin_level',
        'Bone_Marrow_Blasts': 'Bone_Marrow_Blasts',
        'Leukemia_Status': 'Leukemia_Status'
    })
    
    facts['PatientId'] = facts['PatientId'].map(patient_ids)
    facts['RegionId'] = facts['Country'].map(region_ids)
    return facts.drop(columns=['Country'])

In [ ]:
def extract_patient_info(df: pd.DataFrame, medical_history_ids: Dict[Any, int]) -> pd.DataFrame:
    """
    Extracts and transforms data for PatientInfo dimension
    
    Args:
        df: Source DataFrame containing raw data
        medical_history_ids: Mapping dictionary for medical history IDs
        
    Returns:
        Processed DataFrame for Dim_PatientInfo
    """
    df['medical_key'] = df[
        ['genetic_mutation', 'family_history', 'smoking_status',
         'alcohol_consumption', 'radiation_exposure', 'infection_history',
         'chronic_illness', 'immune_disorders']
    ].astype(str).apply(lambda x: '|'.join(x), axis=1)
    
    patient_info = df[[
        'id', 'age', 'gender', 'socioeconomic_status', 'urban_rural', 'medical_key'
    ]].copy().rename(columns={
        'id': 'Patient_id',
        'urban_rural': 'Urban_Rural',
        'socioeconomic_status': 'Socioeconomic_Status'
    })
    
    patient_info['MedicalHistoryId'] = patient_info['medical_key'].map(medical_history_ids)
    return patient_info.drop(columns=['medical_key'])


In [23]:
def process_dimensions(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """
    Orchestrates the complete dimension processing pipeline
    
    Args:
        df: Source DataFrame containing raw data
        
    Returns:
        Dictionary of processed DataFrames for all dimensions
    """

    dim_medical = extract_medical_history(df)
    dim_region = extract_region(df)
    

    medical_keys = df[
        ['Genetic_Mutation', 'Family_History', 'Smoking_Status',
         'Alcohol_Consumption', 'Radiation_Exposure', 'Infection_History',
         'Chronic_Illness', 'Immune_Disorders']
    ].astype(str).apply(lambda x: '|'.join(x), axis=1)
    
    medical_history_ids = dict(zip(medical_keys.unique(), dim_medical['MedicalHistoryId']))
    patient_ids = dict(zip(df['Patient_ID'], df['Patient_ID']))  # Assuming Patient_ID is unique
    region_ids = dict(zip(dim_region['Country'], dim_region['RegionId']))
    

    dim_patient = extract_patient_info(df, medical_history_ids)
    fact_leukemia = extract_leukemia_facts(df, patient_ids, region_ids)
    
    return {
        'Dim_MedicalHistory': dim_medical,
        'Dim_Region': dim_region,
        'Dim_PatientInfo': dim_patient,
        'Fact_Leukemia': fact_leukemia
    }


In [31]:
def load_db_credentials(json_path: str) -> Dict[str, str]:
    """
    Loads database credentials from JSON file
    
    Args:
        json_path: Path to JSON credentials file
        
    Returns:
        Dictionary with connection parameters
        
    Raises:
        ValueError: If JSON file is invalid or missing required keys
    """
    try:
        with open(json_path, 'r') as f:
            creds = json.load(f)
            
        # Verify required keys are present
        required_keys = ['db_host', 'db_name', 'db_user', 'db_password']
        if not all(key in creds for key in required_keys):
            missing = set(required_keys) - set(creds.keys())
            raise ValueError(f"Missing required keys in credentials: {missing}")
            
        return creds
        
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format in credentials file")
    except FileNotFoundError:
        raise ValueError(f"Credentials file not found at {json_path}")

In [26]:
def export_to_postgres(df_dict: Dict[str, pd.DataFrame], creds: Dict[str, str], schema: str = None) -> None:
    """
    Exports all DataFrames to PostgreSQL database
    
    Args:
        df_dict: Dictionary of {table_name: DataFrame} to export
        creds: Database credentials dictionary
        schema: Target schema name (optional)
        
    Raises:
        RuntimeError: If export fails
    """
    try:
  
        engine = create_engine(
            f"postgresql://{creds['db_user']}:{creds['db_password']}@"
            f"{creds['db_host']}:5432/{creds['db_name']}",
            connect_args={'connect_timeout': 10}
        )
        
   
        with engine.connect() as conn:
            print("Successfully connected to PostgreSQL database")
        

        for table_name, df in df_dict.items():
            try:
                df.to_sql(
                    name=table_name.lower(),
                    con=engine,
                    schema=schema,
                    if_exists='replace',
                    index=False,
                    chunksize=1000,
                    method='multi'
                )
                print(f"Successfully exported {table_name} ({len(df)} records)")
            except Exception as e:
                print(f"Failed to export {table_name}: {str(e)}")
                continue
                
    except Exception as e:
        raise RuntimeError(f"Database export failed: {str(e)}")
    finally:
        if 'engine' in locals():
            engine.dispose()
            print("Database connection closed")

In [27]:
def main(data_path: str, creds_path: str) -> None:
    """
    Main execution function for complete ETL pipeline
    
    Args:
        data_path: Path to source data file
        creds_path: Path to credentials JSON file
    """
    try:
        print(f"Loading data from {data_path}")
        df = pd.read_csv(data_path)
        
        print("Processing data dimensions...")
        dimensions = process_dimensions(df)
     
        print(f"Loading credentials from {creds_path}")
        creds = load_db_credentials(creds_path)
        

        print("Exporting to PostgreSQL database...")
        export_to_postgres(dimensions, creds)
        
        print("ETL pipeline completed successfully!")
        
    except Exception as e:
        print(f"ETL pipeline failed: {str(e)}")
        raise


In [33]:
DATA_FILE = "/home/user/leucemia/Leukemia-Cancer-Risk-ETL/data/biased_leukemia_dataset.csv"
CREDS_FILE = "/home/user/leucemia/Leukemia-Cancer-Risk-ETL/credentialsdb.json"

main(DATA_FILE, CREDS_FILE)

Loading data from /home/user/leucemia/Leukemia-Cancer-Risk-ETL/data/biased_leukemia_dataset.csv
Processing data dimensions...
Loading credentials from /home/user/leucemia/Leukemia-Cancer-Risk-ETL/credentialsdb.json
Exporting to PostgreSQL database...
Successfully connected to PostgreSQL database
Successfully exported Dim_MedicalHistory (255 records)
Successfully exported Dim_Region (22 records)
Successfully exported Dim_PatientInfo (143194 records)
Successfully exported Fact_Leukemia (143194 records)
Database connection closed
ETL pipeline completed successfully!
